# Repozytorium - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu **Repozytorium** w aplikacji.

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExeWR3YThvcTgxcDVhbzl2MzFja2hmcG9mMmc0d2s1aXpicDA5eHlsaSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/b2E5YdNOMy3IC3fVU8/giphy.gif" width="200" />

**Repozytorium** jest warstwą pośredniczącą między danymi a resztą aplikacji, zapewniającymi dostęp do źródeł danych (np. bazy danych, zdalne API, odczyt z pliku) oraz operacje na danych. W kontekście wzorca `MVVM`, repozytorium pomaga w oddzieleniu logiki biznesowej od interakcji z danymi, co ułatwia testowanie, utrzymanie i rozszerzanie aplikacji. 
- Separacja odpowiedzialności - Izoluje logikę dostępu do danych od logiki biznesowej i ui. To pozwala na łatwiejsze zarządzanie i utrzymanie kodu, a także ułatwia współpracę między różnymi zespołami programistycznymi.
- Testowalność - Zapewnia łatwość testowania, ponieważ zapewnia abstrakcję nad danymi. Dzięki temu możemy tworzyć testy jednostkowe i testy integracyjne, które pozwalają nam weryfikować poprawność funkcjonalności bez konieczności dostępu do rzeczywistych źródeł danych.
- Łatwa wymiana źródeł danych - Dzięki repozytorium możemy zmieniać źródło danych bez wprowadzania zmian w innych częściach aplikacji. Na przykład, jeśli nasza aplikacja korzysta z lokalnej bazy danych, ale chcemy w przyszłości przejść na zdalne API, możemy to zrobić bez konieczności modyfikowania `ViewModel`'ów i warstwy ui.
- Obsługa błędów i odzyskiwanie - Repozytoria mogą obsługiwać błędy związane z danymi, takie jak problemy z siecią czy błędy związane z bazą danych.
- Optymalizacja dostępu do danych - Repozytoria mogą wprowadzać mechanizmy optymalizacji dostępu do danych, np. *cache'owanie* wyników zapytań czy ograniczenie ilości komunikacji sieciowej. To może przyspieszyć działanie aplikacji i zmniejszyć zużycie zasobów urządzenia. (*cachowanie* pojawi się w ostatnim module zajęć)
- Logika dostępu do danych - jeżeli mamy kilka źródeł danych, logikę dostępu (dostęp do API co określony czas, na żądanie użytkownika, w przeciwnym wypadku wczytanie danych z lokalnej bazy) możemy umieścić w repozytorium, pozostawiając `ViewModel` z jednym źródłem danych

Stosowanie repozytorium nie jest obowiązkowe we wzorcu MVVM (nie jest jego częścią), ale jest jednym z wielu sposobów na zorganizowanie kodu w bardziej czytelny sposób, co ułatwia pracę nad większymi i bardziej skomplikowanymi projektami. 

Do wykorzystania repozytorium nie są wymagane żadne dodatkowe zależności w projekcie, dodajmy viewmodel

do bloku `dependencies`
```kotlin
    ...
    implementation (libs.lifecycle.viewmodel)
    implementation(libs.lifecycle.viewmodel.android)
    ...
```

do pliku `libs.versions.toml`

```kotlin
    [versions]
    ...
    lifecycleViewmodelAndroid = "2.8.4"
    
    [libraries]
    ...
    lifecycle-viewmodel = { module = "androidx.lifecycle:lifecycle-viewmodel", version.ref = "lifecycleViewmodel" }
    lifecycle-viewmodel-android = { group = "androidx.lifecycle", name = "lifecycle-viewmodel-android", version.ref = "lifecycleViewmodelAndroid" }
    ...
```

Nasza aplikacja będzie wyświetlała listę użytkowników, którą wygenerujemy (*dummy data*). Dodajmy model, oraz dane.

In [ ]:
public class User {
    private String firstName;
    private String lastName;

    public User(String firstName, String lastName) {
        this.firstName = firstName;
        this.lastName = lastName;
    }

    public String getFirstName() {
        return firstName;
    }

    public void setFirstName(String firstName) {
        this.firstName = firstName;
    }

    public String getLastName() {
        return lastName;
    }

    public void setLastName(String lastName) {
        this.lastName = lastName;
    }

    @Override
    public boolean equals(Object o) {
        if (this == o) return true;
        if (o == null || getClass() != o.getClass()) return false;
        User user = (User) o;
        return Objects.equals(firstName, user.firstName) &&
                Objects.equals(lastName, user.lastName);
    }

    @Override
    public int hashCode() {
        return Objects.hash(firstName, lastName);
    }
}


In [ ]:
public final class DataProvider {

    private DataProvider(){}
    private static final List<String> firstNames = Arrays.asList(
            "Adam", "Ewa", "Jan", "Anna", "Piotr", "Maria", "Tomasz", "Małgorzata", "Krzysztof", "Alicja",
            "Andrzej", "Joanna", "Michał", "Barbara", "Kamil", "Magdalena", "Robert", "Monika", "Mateusz", "Natalia"
    );

    private static final List<String> lastNames = Arrays.asList(
            "Nowak", "Kowalski", "Wiśniewski", "Wójcik", "Kowalczyk", "Kamiński", "Lewandowski", "Zieliński", "Szymański",
            "Woźniak", "Dąbrowski", "Kozłowski", "Jankowski", "Mazur", "Kwiatkowski", "Krawczyk", "Piotrowski", "Grabowski",
            "Nowakowski", "Pawłowski"
    );

    private static final Random random = new Random();

    public static List<User> getUsers() {
        return generateUsers(41);
    }

    private static List<User> generateUsers(int count) {
        List<User> users = new ArrayList<>();
        for (int i = 0; i < count; i++) {
            String firstName = getRandomElement(firstNames);
            String lastName = getRandomElement(lastNames);
            User user = new User(firstName, lastName);
            users.add(user);
        }
        return users;
    }

    private static String getRandomElement(List<String> list) {
        int index = random.nextInt(list.size());
        return list.get(index);
    }
}

Dodajmy repozytorium, jest to zwykła klasa.

In [ ]:
public class UserRepository {
    public List<User> getUsers(){
        return DataProvider.getUsers();
    }
}

Ten kod reprezentuje klasę o nazwie `UserRepository`, która jest odpowiedzialna za dostarczanie listy użytkowników (`User`) z źródła danych.

- `public List<User> getUsers(){ ... }` - Funkcja zwracająca listę użytkowników.

W następnym kroku dodajmy `ViewModel`

In [ ]:
public class UserViewModel extends ViewModel {

    private final UserRepository userRepository;
    private final MutableLiveData<List<User>> usersList = new MutableLiveData<>();
    public LiveData<List<User>> getUsersList() {
        return usersList;
    }

    public UserViewModel() {
        userRepository = new UserRepository();
        loadUsers();
    }

    private void loadUsers(){
        usersList.setValue(userRepository.getUsers());
    }

}

`UserViewModel` służy jako warstwa pośrednicząca między ui a repozytorium. Po utworzeniu obiektu, inicjalizuje się funkcję `loadUsers()`, która asynchronicznie pobiera listę użytkowników z repozytorium i przechowuje je w `usersList`.

- `public UserViewModel() { ... }` - W konstruktorze tworzona jest instancja repozytorium, oraz wywoływana jest funkcja `loadUsers()`, która ma za zadanie załadować listę użytkowników z repozytorium i umieścić je w `usersList`.
- `viewModelScope.launch { ... }` - `viewModelScope` jest dostarczany przez `ViewModel` i automatycznie zarządza cyklem życia Coroutine w zależności od cyklu życia `ViewModel`. `userRepository.getUsers()` wewnątrz bloku `launch` asynchronicznie pobiera listę użytkowników.

Utwórzmy Fragment z `RecyclerView` w celu wyświetlenia listy.

In [ ]:
public class UserViewHolder extends RecyclerView.ViewHolder {
    private RvItemBinding binding;

    public UserViewHolder(RvItemBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(User item) {
        String fullName = item.getFirstName() + " " + item.getLastName();
        binding.userTextView.setText(fullName);
    }
}

In [ ]:
public class UserComparator extends DiffUtil.ItemCallback<User> {
    @Override
    public boolean areItemsTheSame(@NonNull User oldItem, @NonNull User newItem) {
        return oldItem == newItem;
    }

    @Override
    public boolean areContentsTheSame(User oldItem, @NonNull User newItem) {
        return oldItem.equals(newItem);
    }
}

In [ ]:
public class UserAdapter extends ListAdapter<User, UserViewHolder> {
    public UserAdapter(@NonNull DiffUtil.ItemCallback<User> diffCallback) {
        super(diffCallback);
    }

    @NonNull
    @Override
    public UserViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        LayoutInflater inflater = LayoutInflater.from(parent.getContext());
        RvItemBinding binding = RvItemBinding.inflate(inflater, parent, false);
        return new UserViewHolder(binding);
    }

    @Override
    public void onBindViewHolder(@NonNull UserViewHolder holder, int position) {
        User item = getItem(position);
        holder.bind(item);
    }
}

In [ ]:
public class UserFragment extends Fragment {

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        com.example.repositorybasicsjava.databinding.FragmentUserBinding binding = FragmentUserBinding.inflate(inflater);

        UserViewModel viewModel = new ViewModelProvider(this).get(UserViewModel.class);
        UserAdapter userAdapter = new UserAdapter(new UserComparator());

        viewModel.getUsersList().observe(getViewLifecycleOwner(), users -> userAdapter.submitList(new ArrayList<>(users)));

        binding.rvList.setAdapter(userAdapter);
        binding.rvList.setLayoutManager(new LinearLayoutManager(requireContext()));

        return binding.getRoot();
    }
}

Możemy przetestować aplikację

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExeWR3YThvcTgxcDVhbzl2MzFja2hmcG9mMmc0d2s1aXpicDA5eHlsaSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/b2E5YdNOMy3IC3fVU8/giphy.gif" width="200" />